In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import DenseNet121
from sklearn.model_selection import train_test_split

In [5]:
pip install pandas

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     ---------------------------------------- 0.0/341.8 kB ? eta -:--:--
     ---- ----------------------------------- 41.0/341.8 kB ? eta -:--:--
     ------------- ------------------------ 122.9/341.8 kB 1.4 MB/s eta 0:00:01
     --------------------- ---------------- 194.6/341.8 kB 1.7 MB/s eta 0:00:01
     -------------------------------------- 341.8/341.8 kB 1.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
    --------------------------------------- 0.2/10.6 MB 5.9 MB/s eta 0:00:02
    --------------------------------------- 0.3/10.6 MB 2.6 MB/s eta 0:00:04
   - -------------------------------------- 0.3/10.6 MB 2.3 MB/s eta 0:00:05
   - -------------------------------------- 0.4/10.6 MB 2.3 MB/s eta 0:00:05
   - -------------------------------------- 0.4/10.6 MB 2.3 MB/s eta 0:00:05
   - -------------------------------------- 0.5/10.6 MB 1.7 MB/s eta 0:00:06
   - ---

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import DenseNet121
from sklearn.model_selection import train_test_split

# Define your dataset path
dataset_path = ',/Kle_Tech/BE/Programs/Python ML/train'

# List all image files in the dataset path
image_files = [os.path.join(dataset_path, file) for file in os.listdir(
    dataset_path) if file.endswith('.jpg')]

# Assuming a binary classification (e.g., healthy vs. diseased eyes)
labels = [0 if 'healthy' in file else 1 for file in image_files]

# Calculate a reasonable test size based on the size of your dataset
dataset_size = len(image_files)

# Adjust test size to ensure there's enough data for training
test_size = min(0.2, dataset_size - 1) if dataset_size > 1 else 0.2

# If the dataset size is very small, set a default test size
if dataset_size <= 1:
    test_size = 0.1

# Split the data into training and testing sets
train_files, test_files, train_labels, test_labels = train_test_split(
    image_files, labels, test_size=test_size, random_state=42)


# Function to load and preprocess images
def load_and_preprocess(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)  # Assuming RGB images
    # Resize images to a consistent size
    img = tf.image.resize(img, (224, 224))
    img = img / 255.0  # Normalize pixel values to the range [0, 1]
    return img, label


# Create TensorFlow Datasets
train_dataset = tf.data.Dataset.from_tensor_slices((train_files, train_labels))
train_dataset = train_dataset.map(load_and_preprocess)
test_dataset = tf.data.Dataset.from_tensor_slices((test_files, test_labels))
test_dataset = test_dataset.map(load_and_preprocess)

# Shuffle and batch the datasets
batch_size = 32
train_dataset = train_dataset.shuffle(
    buffer_size=len(train_files)).batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

# Load the pre-trained DenseNet121 model
base_model = DenseNet121(
    include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Add custom classification layers on top
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    # Binary classification, so using sigmoid activation
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_dataset, epochs=10, validation_data=test_dataset)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_dataset)
print(f'Test accuracy: {test_acc}')

# Save the model
model.save('eye_disease_detection.h5')

ModuleNotFoundError: No module named 'sklearn'